# Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import wilcoxon
from collections import defaultdict

path="..\data"

import sys
sys.path.append(path+'\..\src')
import deferred_acceptance_algorithm
from deferred_acceptance_algorithm import *

# Create the functions

In [ ]:
def create_data(num_proposers=10, num_reviewers=10, outside_option="random", individual_rationality=True):
    proposers = [int(a) for a in list(range(1, num_proposers+1))]
    reviewers = [int(a) for a in list(range(num_proposers+1, num_proposers+num_reviewers+1))]
    
    proposer_prefs={}
    for p in proposers:
        proposer_prefs[p] = random.sample(reviewers, len(reviewers))
    reviewer_prefs={}
    for r in reviewers:
        reviewer_prefs[r] = random.sample(proposers, len(proposers))
        
    if individual_rationality:
        if outside_option=="random":
            for p in proposer_prefs:
                proposer_prefs[p].insert(random.choice(list(range(0, num_reviewers))), p)
            for r in reviewer_prefs:
                reviewer_prefs[r].insert(random.choice(list(range(0, num_proposers))), r)
        elif outside_option in range(0,num_reviewers+1):
            for p in proposer_prefs:
                proposer_prefs[p].insert(outside_option, p)
            for r in reviewer_prefs:
                reviewer_prefs[r].insert(outside_option, r)
    print(f"Prefs: , {proposer_prefs}, {reviewer_prefs}")
    return proposer_prefs, reviewer_prefs

def proposer_without_match(matches, proposers):
    for proposer in proposers:
        if proposer not in matches:
            return proposer

def deferred_acceptance(proposer_prefs, reviewer_prefs, individual_rationality=True):
    all = list(proposer_prefs.keys()) + list(reviewer_prefs.keys())
    if not individual_rationality:
        reviewer_queue = defaultdict(int)
        proposers = list(proposer_prefs.keys())
        matches = {}
        while True:
            proposer = proposer_without_match(matches, proposers)
            if not proposer:
                break
            
            reviewer_index = reviewer_queue[proposer]
            reviewer_queue[proposer] += 1
    
            try:
                reviewer = proposer_prefs[proposer][reviewer_index]
            except IndexError:
                matches[proposer] = proposer
                continue
    
            print(f'Trying {proposer} with {reviewer}... ' , end='')
            
            prev_proposer = matches.get(reviewer, None)
    
            if not prev_proposer:
                matches[proposer] = reviewer
                matches[reviewer] = proposer
                print('deferred')
                
            elif reviewer_prefs[reviewer].index(proposer) < \
                 reviewer_prefs[reviewer].index(prev_proposer):
                matches[proposer] = reviewer
                matches[reviewer] = proposer
                del matches[prev_proposer]
                print('matched')
                
            else:
                print('rejected')
                
        matched_pair = {proposer: matches[proposer] for proposer in proposer_prefs.keys()}
        for i in list(matched_pair.keys()):
            if matched_pair[i] == i:
                del matched_pair[i]
        matched_individuals = list(matched_pair.keys()) + list(matched_pair.values())
        unmatched_individuals = [p for p in all if p not in matched_individuals]
        for u in unmatched_individuals:
            matches[u] = u
                
    if individual_rationality:
        reviewer_queue = defaultdict(int)
        proposers = list(proposer_prefs.keys())
        matches = {}
    
        while True:
            # Store a proposer from unmatched proposers
            proposer = proposer_without_match(matches, proposers)
            if not proposer:
                break
    
            # Store reviewer index
            reviewer_index = reviewer_queue[proposer]
            reviewer_queue[proposer] += 1
        
            try: 
                reviewer = proposer_prefs[proposer][reviewer_index]
            except IndexError:
                matches[proposer] = proposer
                continue
        
            proposer_self = proposer_prefs[proposer].index(proposer)
            if proposer_prefs[proposer].index(reviewer) < proposer_self:
                print(f'Trying {proposer} with {reviewer}... ' , end='')
                prev_proposer = matches.get(reviewer, None)
                reviewer_self = reviewer_prefs[reviewer].index(reviewer)
                
                if not prev_proposer:
                    if reviewer_prefs[reviewer].index(proposer) < reviewer_self:
                        matches[proposer] = reviewer
                        matches[reviewer] = proposer
                        print('deferred')
                    elif reviewer_prefs[reviewer].index(proposer) > reviewer_self:
                        print("rejected")
                elif prev_proposer:
                    if reviewer_prefs[reviewer].index(proposer) < reviewer_prefs[reviewer].index(prev_proposer):
                        matches[proposer] = reviewer
                        matches[reviewer] = proposer
                        del matches[prev_proposer]
                        print('deferred')
                    elif reviewer_prefs[reviewer].index(proposer) > reviewer_prefs[reviewer].index(prev_proposer):
                        print("rejected")
                else:
                    print("rejected")
                    
            elif proposer_prefs[proposer].index(reviewer) > proposer_self:
                matches[proposer] = proposer
                del matches[proposer]
        
        matched_pair = {proposer: matches[proposer] for proposer in proposer_prefs.keys()}
        for i in list(matched_pair.keys()):
            if matched_pair[i] == i:
                del matched_pair[i]
        matched_individuals = list(matched_pair.keys()) + list(matched_pair.values())
        unmatched_individuals = [p for p in all if p not in matched_individuals]
        for u in unmatched_individuals:
            matches[u] = u
    print("Matching result: ", {a: matches[a] for a in all})
    return {a: matches[a] for a in all}

def conduct_experiments(num_proposers, num_reviewers, outside_option):
    proposer_prefs, reviewer_prefs = create_data(num_proposers=num_proposers, num_reviewers=num_reviewers, outside_option=outside_option)
   
    res = deferred_acceptance(proposer_prefs, reviewer_prefs)

    proposer_utils=[num_proposers - proposer_prefs[p].index(res[p]) for p in range(1, num_proposers+1)]
    reviewer_utils=[num_reviewers - reviewer_prefs[r].index(res[r]) for r in range(num_proposers+1, num_proposers+num_reviewers+1)]
    total_utils_proposers = np.sum(proposer_utils)
    total_utils_reviewers = np.sum(reviewer_utils)
    print(f"Total utility of proposers: , {total_utils_proposers}")
    print(f"Total utility of reviewers: , {total_utils_reviewers}")
    return proposer_prefs, reviewer_prefs, res, proposer_utils, reviewer_utils, total_utils_proposers, total_utils_reviewers

def iterate_experiments(num_proposers, num_reviewers, num_itr, outside_option):
    random.seed(20210721)

    proposer_prefs_list=[]
    reviewer_prefs_list=[]
    result_list=[]
    proposer_utils_list=[]
    reviewer_utils_list=[]
    total_utils_proposers_list=[]
    total_utils_reviewers_list=[]

    for itr in range(num_itr):
        proposer_prefs, reviewer_prefs, res, proposer_utils, reviewer_utils, total_utils_proposers, total_utils_reviewers = conduct_experiments(num_proposers,num_reviewers, outside_option)
        
        proposer_prefs_list.append(proposer_prefs)
        reviewer_prefs_list.append(reviewer_prefs)
        result_list.append(res)
        proposer_utils_list.append(proposer_utils)
        reviewer_utils_list.append(reviewer_utils)
        total_utils_proposers_list.append(total_utils_proposers)
        total_utils_reviewers_list.append(total_utils_reviewers)

    return proposer_prefs_list, reviewer_prefs_list, result_list, proposer_utils_list, reviewer_utils_list, total_utils_proposers_list, total_utils_reviewers_list

# Small case

In [11]:
num_proposers=10
num_reviewers=10
num_itr=3

proposer_prefs_list, reviewer_prefs_list, result_list, proposer_utils_list, reviewer_utils_list, total_utils_proposers_list, total_utils_reviewers_list = iterate_experiments(num_proposers, num_reviewers, num_itr, 3)
sample = pd.DataFrame({"experiment_id": list(range(1,num_itr+1)), "proposer_prefs": proposer_prefs_list, "reviewer_prefs": reviewer_prefs_list, "matching_result": result_list, "proposer_utils": proposer_utils_list, "reviewer_utils": reviewer_utils_list, "total_utils_proposers": total_utils_proposers_list, "total_utils_reviewers": total_utils_reviewers_list})
display(sample)

Prefs: , {1: [17, 18, 20, 1, 11, 14, 15, 19, 12, 16, 13], 2: [19, 16, 18, 2, 20, 17, 11, 14, 13, 15, 12], 3: [20, 18, 14, 3, 17, 12, 19, 16, 13, 11, 15], 4: [17, 20, 12, 4, 19, 13, 15, 14, 18, 16, 11], 5: [14, 19, 16, 5, 20, 11, 17, 13, 18, 12, 15], 6: [19, 13, 16, 6, 14, 18, 20, 15, 11, 17, 12], 7: [19, 18, 17, 7, 16, 15, 20, 12, 13, 11, 14], 8: [15, 18, 20, 8, 14, 19, 17, 11, 16, 13, 12], 9: [20, 15, 18, 9, 11, 12, 13, 19, 16, 17, 14], 10: [12, 11, 13, 10, 18, 19, 14, 17, 15, 20, 16]}, {11: [10, 4, 5, 11, 9, 1, 3, 2, 6, 7, 8], 12: [1, 7, 10, 12, 3, 4, 6, 5, 8, 9, 2], 13: [5, 9, 2, 13, 4, 3, 8, 10, 1, 7, 6], 14: [5, 9, 10, 14, 1, 8, 3, 4, 7, 2, 6], 15: [2, 6, 3, 15, 10, 8, 9, 1, 5, 7, 4], 16: [1, 2, 4, 16, 9, 8, 10, 5, 3, 7, 6], 17: [8, 10, 9, 17, 6, 2, 4, 3, 1, 7, 5], 18: [8, 6, 7, 18, 10, 9, 3, 4, 5, 1, 2], 19: [4, 8, 6, 19, 3, 2, 7, 9, 5, 1, 10], 20: [8, 2, 5, 20, 9, 6, 4, 1, 3, 10, 7]}
Trying 1 with 17... rejected
Trying 1 with 18... rejected
Trying 1 with 20... rejected
Trying 2 

,experiment_id,proposer_prefs,reviewer_prefs,matching_result,proposer_utils,reviewer_utils,total_utils_proposers,total_utils_reviewers
0,1,"{1: [17, 18, 20, 1, 11, 14, 15, 19, 12, 16, 13...","{11: [10, 4, 5, 11, 9, 1, 3, 2, 6, 7, 8], 12: ...","{1: 1, 2: 16, 3: 3, 4: 4, 5: 14, 6: 19, 7: 7, ...","[7, 9, 7, 7, 10, 10, 7, 9, 7, 10]","[7, 8, 7, 10, 7, 9, 7, 10, 8, 7]",83,80
1,2,"{1: [15, 14, 20, 1, 19, 17, 18, 11, 16, 12, 13...","{11: [3, 8, 7, 11, 9, 5, 6, 2, 4, 1, 10], 12: ...","{1: 1, 2: 2, 3: 11, 4: 12, 5: 5, 6: 6, 7: 18, ...","[7, 7, 10, 8, 7, 7, 10, 7, 10, 10]","[10, 9, 7, 7, 7, 8, 7, 10, 7, 9]",83,81
2,3,"{1: [19, 17, 14, 1, 13, 11, 18, 12, 20, 15, 16...","{11: [8, 10, 5, 11, 3, 6, 9, 2, 1, 7, 4], 12: ...","{1: 19, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 14, 8...","[10, 7, 7, 7, 7, 7, 9, 10, 9, 8]","[10, 7, 7, 10, 7, 7, 8, 9, 8, 7]",81,80


# DA algorithm

In [ ]:
num_proposers=10
num_reviewers=10
num_itr=1000

da_table=pd.DataFrame([])
for i in range(0, 11):
    proposer_prefs_list, reviewer_prefs_list, result_list, proposer_utils_list, reviewer_utils_list, total_utils_proposers_list, total_utils_reviewers_list = iterate_experiments(num_proposers, num_reviewers, num_itr, outside_option=i)
    table = pd.DataFrame({"pref_alone": [i+1]*num_itr, "experiment_id": list(range(1,1+num_itr)), "proposer_prefs": proposer_prefs_list, "reviewer_prefs": reviewer_prefs_list, "matching_result": result_list, "proposer_utils": proposer_utils_list, "reviewer_utils": reviewer_utils_list, "total_utils_proposers": total_utils_proposers_list, "total_utils_reviewers": total_utils_reviewers_list})
    da_table = pd.concat([table, da_table], axis=0)
da_table = da_table.reset_index(drop=True)

da_table.to_pickle(path+"/03_primary/mastertable.pickle")

Streaming output truncated to the last 5000 lines.
Trying 7 with 16... deferred
Trying 5 with 14... deferred
Trying 8 with 14... rejected
Trying 8 with 18... rejected
Trying 8 with 16... rejected
Trying 8 with 19... deferred
Trying 6 with 12... deferred
Trying 9 with 13... rejected
Trying 9 with 18... deferred
Trying 2 with 12... deferred
Trying 6 with 20... deferred
Trying 10 with 17... rejected
Trying 10 with 18... rejected
Trying 10 with 15... rejected
Trying 10 with 13... rejected
Trying 10 with 16... rejected
Trying 10 with 12... rejected
Trying 10 with 11... deferred
Matching result:  {1: 15, 2: 12, 3: 17, 4: 13, 5: 14, 6: 20, 7: 16, 8: 19, 9: 18, 10: 11, 11: 10, 12: 2, 13: 4, 14: 5, 15: 1, 16: 7, 17: 3, 18: 9, 19: 8, 20: 6}
Total utility of proposers: , 86
Total utility of reviewers: , 75
Prefs: , {1: [17, 16, 19, 18, 20, 12, 11, 14, 15, 13, 1], 2: [15, 14, 19, 18, 16, 17, 13, 11, 20, 12, 2], 3: [20, 17, 14, 13, 11, 19, 15, 18, 16, 12, 3], 4: [12, 17, 15, 18, 19, 16, 20, 14, 11,